In [0]:
import pandas as pd
import requests 
import time
from pandas.io.json import json_normalize

In [0]:
response = requests.get("https://data.iowa.gov/resource/runv-jsix.json")
crimes = response.json()
df_inst = json_normalize(crimes)
df_inst.head()

,record_id,offender_number,release_date,fiscal_year_released,closure_type,supervision_status,institution_name,offender_first_name,offender_middle_name,offender_last_name,offender_date_of_birth,sex,race_ethnicity,age_at_release,offense_code,offense_class,offense_description,offense_type,offense_subtype,jurisdiction,admission_date,months_served,offender_suffix
0,1.001E+11,1000998,2014-10-17T00:00:00.000,2015,Death,Prison,Anamosa State Penitentiary,Darrell,William,Starkey,1958-04-05T00:00:00.000,Male,White,56,707.3,B Felony,MURDER 2ND - 85%,Violent,Murder/Manslaughter,Montgomery,1997-11-25T00:00:00.000,202.9605263,NaN
1,10013440819,100134,2011-10-03T00:00:00.000,2012,Parole Granted,Prison,Mount Pleasant Correctional Facility,Scott,NaN,Schalla,1964-07-31T00:00:00.000,Male,White,47,714.2(2),D Felony,THEFT 2ND DEGREE,Property,Theft,Johnson,2011-01-18T00:00:00.000,8.486842105,NaN
2,1004441173,10044,2012-09-21T00:00:00.000,2013,Work Release Granted,Prison,Mount Pleasant Correctional Facility,Dale,Laverne,Fleming,1959-11-24T00:00:00.000,Male,White,52,124.401(5)(c),D Felony,Possession of a Controlled Substance - 3rd and...,Drug,Drug Possession,Henry,2012-02-06T00:00:00.000,7.5,NaN
3,10044640276,100446,2010-04-08T00:00:00.000,2010,Discharged - Expiration of Sentence,Prison,Mount Pleasant Correctional Facility,Dawn,Lynn,Dooley,1966-03-22T00:00:00.000,Female,White,44,321J.2(B),Aggravated Misdemeanor,OPER VEH WH INT (OWI) / AGR MISD / 2ND OFF -,Public Order,OWI,Polk,2009-06-24T00:00:00.000,9.473684211,NaN
4,10046742541,100467,2016-06-20T00:00:00.000,2016,Parole Granted,Prison,Anamosa State Penitentiary,Edward,James,Wilkinson,1965-02-21T00:00:00.000,Male,White,51,124.401(5)(c),D Felony,Possession of a Controlled Substance - 3rd and...,Drug,Drug Possession,Polk,2015-06-12T00:00:00.000,12.30263158,NaN


In [0]:
df_inst.columns

Index(['record_id', 'offender_number', 'release_date', 'fiscal_year_released',
       'closure_type', 'supervision_status', 'institution_name',
       'offender_first_name', 'offender_middle_name', 'offender_last_name',
       'offender_date_of_birth', 'sex', 'race_ethnicity', 'age_at_release',
       'offense_code', 'offense_class', 'offense_description', 'offense_type',
       'offense_subtype', 'jurisdiction', 'admission_date', 'months_served',
       'offender_suffix'],
      dtype='object')

In [0]:
df_inst["institution_name"].unique()

array(['Anamosa State Penitentiary',
       'Mount Pleasant Correctional Facility',
       'Newton Correctional Facility',
       'North Central Correctional Facility', 'Iowa State Penitentiary',
       'Fort Dodge Correctional Facility',
       'Clarinda Correctional Facility',
       'Iowa Medical & Classification Center',
       'Iowa Correctional Institution for Women'], dtype=object)

In [0]:
#Created new variable that records the security level of the prison.
df_inst["institution_type"] = df_inst["institution_name"].fillna('0').map({
    'Anamosa State Penitentiary' : "Maximum",
    'Mount Pleasant Correctional Facility' : "Minimum",
    'Newton Correctional Facility' : "Medium",
    'North Central Correctional Facility' : "Minimum",
    'Iowa State Penitentiary' : "Maximum", 
    'Fort Dodge Correctional Facility' : "Medium",
    'Clarinda Correctional Facility' : "Medium",
    'Iowa Medical & Classification Center' : "Medium",
    'Iowa Correctional Institution for Women' : "Minimum",
    "0" : "N/A"

})

In [0]:
df_inst['closure_type'].unique()

array(['Death', 'Parole Granted', 'Work Release Granted',
       'Discharged - Expiration of Sentence', 'Released to OWI Continuum',
       'Work Unit - Transfer from County Jail Contract',
       'Paroled w/Immediate Discharge',
       'Work Unit - Transfer from Violator Program', 'Revoked',
       'Probation Granted - Reconsideration of Sentence',
       'Work Unit - Prison to Corrections Compact',
       'Terminated by Appeal - Conviction Overturned',
       'Paroled to Detainer - INS', 'Work Unit - Field to Field',
       'Paroled to Detainer - Out of State',
       'Work Unit - Prison to Field',
       'Paroled to Detainer - U.S. Marshall', 'none from supervision',
       'Released to Special Sentence',
       'Work Unit - Prison to Residential', 'Released on Bond',
       'Paroled to Detainer - Iowa'], dtype=object)

In [0]:
#Created new variable that records the reason for an inmate leaving prison. Created to replace "closure_type" column and make it more concise.
df_inst['Exit'] = df_inst["closure_type"].fillna('0').map({
    '0' : "N/A",
    'Death' : 'Death',
    'Work Release Granted' : 'Working',
    'Discharged - Expiration of Sentence' : 'End of Sentence',
    'Released to OWI Continuum' : 'Sent to Treatment Program',
    'Work Unit - Transfer from County Jail Contract' : 'Working',
    'Work Unit - Transfer from Violator Program' : 'Working',
    'Revoked' : 'Revoked',
    'Probation Granted - Reconsideration of Sentence' : 'Probation Granted',
    'Work Unit - Prison to Corrections Compact' : 'Working',
    'Terminated by Appeal - Conviction Overturned' : 'Convition Overturned',
    'Work Unit - Field to Field' : 'Working',
    'Work Unit - Prison to Field' : 'Working',
    'none from supervision' : "N/A",
    'Released to Special Sentence' : 'Released: Special Sentence',
    'Work Unit - Prison to Residential' : 'Working',
    'Released on Bond' : 'Released on Bond'
})

df_inst['Exit'].fillna("Parol", inplace = True)

In [0]:
def mapper(date):
  dct = {
      1:"January",
      2:"February",
      3:"March",
      4:"April",
      5:"May",
      6:"June",
      7:"July",
      8:"August",
      9:"September",
      10:"October",
      11:"November",
      12:"September"
  }
  return dct[date.month]


df_inst['release_month'] = pd.to_datetime(df_inst['release_date'], format = "%Y-%m-%d").map(mapper)


In [0]:
#Dropping redudant / useless columns
df_inst.drop(['record_id', 
              'release_date', 
              'closure_type', 
              'offender_first_name',
              'offender_middle_name', 
              'offender_last_name',
              'offender_date_of_birth', 
              'offense_code',
              'offender_suffix'],
              axis=1, inplace=True)

In [0]:
def mapper(months):
  return round(months, 2)

df_inst['months_served'] = (df_inst["months_served"].astype(float)).map(mapper)

In [0]:
# df_inst.drop(columns=["Unnamed: 0"], inplace=True)

def mapper(months):
  return round(months)/12
def rounder(num):
  return round(num)

df_inst['age_at_admission'] = df_inst['age_at_release'].astype(int) - df_inst['months_served'].map(mapper)
df_inst['age_at_admission'] = df_inst['age_at_admission'].map(rounder)

In [0]:
df_inst['offense_type'].fillna("Other", inplace=True)
df_inst['jurisdiction'].fillna("Other", inplace=True)

In [0]:
df_inst["time served"] = df_inst["months_served"]**.5

In [0]:
df_inst.to_csv("dataframe.csv")

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ls

dataframe.csv  drive/  sample_data/


In [0]:
mv dataframe.csv drive/My\ Drive/301Final


In [0]:
# cd ../../

In [0]:
# ls